<a href="https://colab.research.google.com/github/sswoo333/2022-1-capstone2-ssw/blob/main/Detectron2_Car_LP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

초기환경설정

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install pyyaml==5.1
!pip install torch==1.8.0+cu101 torchvision==0.9.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
#install old version of pytorch since detectron2 hasn't released packages for pytorch 1.9

In [ ]:
 !pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
 # After this setp it will ask you to restart the runtime, please do it 

In [ ]:
import torch
import torchvision
import cv2

utils.py

In [ ]:
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.utils.visualizer import Visualizer
from detectron2.config import get_cfg
from detectron2 import model_zoo

from detectron2.utils.visualizer import ColorMode

import random 
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

#####
from detectron2.engine import HookBase
from detectron2.data import build_detection_train_loader
import detectron2.utils.comm as comm
#####

def plot_samples(dataset_name, n=1):
    dataset_custom = DatasetCatalog.get(dataset_name)
    dataset_custom_metadata = MetadataCatalog.get(dataset_name)

    for s in random.sample(dataset_custom, n):
        img = cv2.imread(s["file_name"])
        v = Visualizer(img[:,:,::-1], metadata=dataset_custom_metadata, scale=1)
        v = v.draw_dataset_dict(s)
        plt.figure(figsize=(14,10))
        plt.imshow(v.get_image())
        plt.show()

def get_train_cfg(config_file_path, checkpoint_url, train_datasetname, test_dataset_name, num_classes, device, output_dir):
    cfg = get_cfg()

    cfg.merge_from_file(model_zoo.get_config_file(config_file_path))
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(checkpoint_url)
    cfg.DATASETS.TRAIN = (train_dataset_name,)
    cfg.DATASETS.TEST = (test_dataset_name,)  #  Validation 

    #####
    cfg.DATASETS.VAL = (test_dataset_name,)
    #####

    """cfg.TEST.EVAL_PERIOD = 100 # 1000 iteration 중에 100번마다 validation 수행 """

    cfg.DATALOADER.NUM_WORKERS = 1      # class의 개수 

    cfg.SOLVER.IMS_PER_BATCH = 2
    cfg.SOLVER.BASE_LR = 0.00025    # Learning rate
    cfg.SOLVER.MAX_ITER = 5000      # 학습횟수 iteration 
    cfg.SOLVER.STEPS = []

    cfg.MODEL.ROI_HEADS.NUM_CLASSES = num_classes
    cfg.MODEL.DEVICE = device
    cfg.OUPUT_DIR = output_dir

    return cfg

#####


class ValidationLoss(HookBase):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg.clone()
        self.cfg.DATASETS.TRAIN = cfg.DATASETS.VAL
        self._loader = iter(build_detection_train_loader(self.cfg))
        
    def after_step(self):
        data = next(self._loader)
        with torch.no_grad():
            loss_dict = self.trainer.model(data)
            
            losses = sum(loss_dict.values())
            assert torch.isfinite(losses).all(), loss_dict

            loss_dict_reduced = {"val_" + k: v.item() for k, v in 
                                 comm.reduce_dict(loss_dict).items()}
            losses_reduced = sum(loss for loss in loss_dict_reduced.values())
            if comm.is_main_process():
                self.trainer.storage.put_scalars(total_val_loss=losses_reduced, 
                                                 **loss_dict_reduced)
#####

def on_image(image_path, predictor) :
    im = cv2.imread(image_path)
    outputs = predictor(im)
    v = Visualizer(im[:,:,::-1], metadata={}, scale=1, instance_mode=ColorMode.SEGMENTATION)
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))

    plt.figure(figsize=(14,10))
    plt.imshow(v.get_image())
    plt.show()

def on_video(videoPath, predictor):
    cap = cv2.VideoCapture(videoPath)
    if (cap.isOpened()==False):
        print("Error opening file....")
        return

    (success,image) = cap.read()
    while success:
        predictions = predictor(image)
        v = Visualizer(image[:,:,::-1], metadata={}, scale=1, instance_mode=ColorMode.SEGMENTATION)
        output = v.draw_instance_predictions(predictions["instances"].to("cpu"))

        cv2.imshow("Result", output.get_image()[:,:,::-1])

        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break
        (success,image) = cap.read()


train.py

In [ ]:
from detectron2.utils.logger import setup_logger

setup_logger()

from detectron2.data.datasets import register_coco_instances 
from detectron2.engine import DefaultTrainer


### 모델평가 관련
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader


import os
import pickle

#from utils import *

config_file_path = "COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml" #COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml
checkpoint_url = "COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"

output_dir = "./output/object_detection" 
num_classes = 1 

device = "cuda" # "cuda" or "cpu"

train_dataset_name = "LP_train"
train_images_path = "train"
train_json_annot_path = "train.json"

test_dataset_name = "LP_test"
test_images_path = "test"
test_json_annot_path = "test.json"

cfg_save_path = "OD_cfg.pickle"

##############################################

if train_dataset_name in DatasetCatalog.list():
    DatasetCatalog.remove(train_dataset_name)

if test_dataset_name in DatasetCatalog.list():
    DatasetCatalog.remove(test_dataset_name)

##################################################

register_coco_instances(name = train_dataset_name, metadata={},
json_file= train_json_annot_path, image_root=train_images_path)

register_coco_instances(name = test_dataset_name, metadata={},
json_file= test_json_annot_path, image_root=test_images_path)

plot_samples(dataset_name=train_dataset_name, n=2)

################################################

def main():
    cfg = get_train_cfg(config_file_path, checkpoint_url, train_dataset_name, test_dataset_name, num_classes, device, output_dir)

    with open(cfg_save_path, 'wb') as f :
        pickle.dump(cfg,f,protocol=pickle.HIGHEST_PROTOCOL)

    os.makedirs(cfg.OUTPUT_DIR,exist_ok=True)
    trainer = DefaultTrainer(cfg)
    #####
    val_loss = ValidationLoss(cfg)  
    trainer.register_hooks([val_loss])
    # swap the order of PeriodicWriter and ValidationLoss
    trainer._hooks = trainer._hooks[:-2] + trainer._hooks[-2:][::-1]
    #####
    
    trainer.resume_or_load(resume=False)

    trainer.train()  # 학습 시작 

    ### 모델 평가(mAP)
    evaluator = COCOEvaluator("LP_test", cfg, False, output_dir="./output/")
    val_loader = build_detection_test_loader(cfg, "LP_test")
    print(inference_on_dataset(trainer.model, val_loader, evaluator))  
    ###

if __name__ == '__main__' :
    main()
    
###################################################


# **TensorBoard** Loss function graph


In [ ]:
%load_ext tensorboard
%tensorboard --logdir output

test.py

In [ ]:
from detectron2.engine import DefaultPredictor

import os
import pickle

#from utils import *

cfg_save_path = "OD_cfg.pickle"

with open(cfg_save_path,'rb') as f:
    cfg = pickle.load(f)

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5

predictor = DefaultPredictor(cfg)

image_path = "test_L/Cars408.png"
#videoPath = "test/highway.mp4"

on_image(image_path, predictor)
#on_video(videoPath, predictor)

